In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [3]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [4]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [5]:
# 생성기와 판별기
# cost function : 확률적 함수
# MSE, Enthropy 분류
# KL-Divergence : GAN, VAE - 분포의 차를 확인하는 함수
# 생성기 -> noise가 input 데이터, cost function : KL-Divergence

def generator(noise_z) :
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

def discriminator(inputs) :
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2)+D_b2)
    return output

def get_noise(batch_size, n_noise) :
    return np.random.normal(size = (batch_size, n_noise))
    

In [6]:
G = generator(Z) # 노이즈 => 이미지 생성
D_gene = discriminator(G)
D_real = discriminator(X)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

In [7]:
%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [13]:
for epoch in range(total_epoch) :
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X: batch_xs, Z : noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict ={Z:noise})
    print('Epoch :', '%04d' %epoch,
         'D loss :', '{:.4}'.format(loss_val_D),
         'G loss :', '{:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0 :
        sample_size = 10
        # 노이즈로부터 이미지를 생성 'generator'
        noise = get_noise(sample_size, n_noise) # 10x128
        samples = sess.run(G, feed_dict = {Z:noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1)) 
        for i in range(sample_size) :
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print("최적화완료")

Epoch : 0000 D loss : -0.6552 G loss : -2.252
Epoch : 0001 D loss : -0.4328 G loss : -2.392
Epoch : 0002 D loss : -0.6142 G loss : -2.268
Epoch : 0003 D loss : -0.752 G loss : -2.152
Epoch : 0004 D loss : -0.452 G loss : -2.128
Epoch : 0005 D loss : -0.5034 G loss : -2.56
Epoch : 0006 D loss : -0.4963 G loss : -2.522
Epoch : 0007 D loss : -0.6778 G loss : -2.16
Epoch : 0008 D loss : -0.7746 G loss : -2.001
Epoch : 0009 D loss : -0.6461 G loss : -2.161
Epoch : 0010 D loss : -0.7596 G loss : -1.904
Epoch : 0011 D loss : -0.7788 G loss : -1.901
Epoch : 0012 D loss : -0.7931 G loss : -1.914
Epoch : 0013 D loss : -0.7943 G loss : -1.941
Epoch : 0014 D loss : -0.816 G loss : -1.588
Epoch : 0015 D loss : -0.8008 G loss : -1.817
Epoch : 0016 D loss : -0.715 G loss : -1.824
Epoch : 0017 D loss : -0.5793 G loss : -2.245
Epoch : 0018 D loss : -0.6983 G loss : -1.867
Epoch : 0019 D loss : -0.8668 G loss : -2.175
Epoch : 0020 D loss : -0.7225 G loss : -2.001
Epoch : 0021 D loss : -0.7079 G loss : -